In [0]:
from google.colab import files
files.upload()

{}

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p /root/.kaggle
!cp /content/kaggle.json /root/.kaggle

In [0]:
! kaggle datasets download -d cfpb/us-consumer-finance-complaints

us-consumer-finance-complaints.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!unzip /content/us-consumer-finance-complaints.zip

Archive:  /content/us-consumer-finance-complaints.zip
replace consumer_complaints.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [0]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,LSTM,Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import utils as np_utils

In [0]:
data = pd.read_csv("/content/consumer_complaints.csv")
data.keys()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zipcode', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response', 'consumer_disputed?', 'complaint_id'],
      dtype='object')

In [0]:
data = pd.read_csv("/content/consumer_complaints.csv", 
                usecols=('product','consumer_complaint_narrative'),
                dtype={'consumer_complaint_narrative': object})

data=data[data['consumer_complaint_narrative'].notnull()]

data=data[data['product'].notnull()]
data.reset_index(drop=True,inplace=True)
x = data.iloc[:, 1].values
y = data.iloc[:, 0].values
print(y)

['Debt collection' 'Consumer Loan' 'Mortgage' ... 'Payday loan' 'Mortgage'
 'Mortgage']


In [0]:
print(np.unique(y, return_counts=True))

(array(['Bank account or service', 'Consumer Loan', 'Credit card',
       'Credit reporting', 'Debt collection', 'Money transfers',
       'Mortgage', 'Other financial service', 'Payday loan',
       'Prepaid card', 'Student loan'], dtype=object), array([ 5711,  3678,  7929, 12526, 17552,   666, 14919,   110,   726,
         861,  2128]))


In [0]:
tokenizer = Tokenizer(num_words= 200, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)
x = sequence.pad_sequences(x, maxlen=200)

print(x)

[[  0   0   0 ...   3  84 108]
 [  0   0   0 ...   2   8   6]
 [145  10 112 ...   7   9   7]
 ...
 [  0   0   0 ... 171  66   1]
 [  0   0   0 ...   2 150  68]
 [ 32   2   4 ...   5  24  16]]


In [0]:
labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)
print(y)

[4 1 6 ... 8 6 6]


In [0]:
print(np.unique(y, return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 5711,  3678,  7929, 12526, 17552,   666, 14919,   110,   726,
         861,  2128]))


In [0]:
y = np_utils.to_categorical(y, num_classes= 11)

print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
np.random.seed(10)
indices = np.arange(len(x))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [0]:
index_from=3
start_char = 1
if start_char is not None:
        x = [[start_char] + [w + index_from for w in x1] for x1 in x]
elif index_from:
        x = [[w + index_from for w in x1] for x1 in x]

In [0]:
num_words = None
if not num_words:
        num_words = max([max(x1) for x1 in x])
        
oov_char = 2
skip_top = 0

if oov_char is not None:
        x = [[w if (skip_top <= w < num_words) else oov_char for w in x1] for x1 in x]
else:
        x = [[w for w in x1 if (skip_top <= w < num_words)] for x1 in x]
        
test_split = 0.2
idx = int(len(x) * (1 - test_split))
x_train, y_train = np.array(x[:idx]), np.array(y[:idx])
x_test, y_test = np.array(x[idx:]), np.array(y[idx:])

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(53444, 201)
(53444, 11)
(13362, 201)
(13362, 11)


In [0]:
x_train = sequence.pad_sequences(x_train, maxlen=201)
x_test = sequence.pad_sequences(x_test, maxlen=201)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (53444, 201)
x_test shape: (13362, 201)


In [0]:
max_features = 1000
maxlen = 201
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250

model = Sequential()
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(11))
model.add(Activation('softmax'))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 201, 50)           50000     
_________________________________________________________________
dropout_2 (Dropout)          (None, 201, 50)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 199, 250)          37750     
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_3 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 250)              

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model.fit(x_train, 
          y_train,
          batch_size=256,
          epochs=15,
          validation_data=(x_test, y_test))

Epoch 1/15
209/209 [==============================] - 3s 14ms/step - loss: 0.9745 - accuracy: 0.6914 - val_loss: 0.9226 - val_accuracy: 0.7181
Epoch 2/15
209/209 [==============================] - 3s 13ms/step - loss: 0.9393 - accuracy: 0.7015 - val_loss: 0.9347 - val_accuracy: 0.7079
Epoch 3/15
209/209 [==============================] - 3s 13ms/step - loss: 0.9396 - accuracy: 0.7026 - val_loss: 0.9153 - val_accuracy: 0.7145
Epoch 4/15
209/209 [==============================] - 3s 13ms/step - loss: 0.9237 - accuracy: 0.7080 - val_loss: 0.9278 - val_accuracy: 0.7170
Epoch 5/15
209/209 [==============================] - 3s 13ms/step - loss: 0.9240 - accuracy: 0.7077 - val_loss: 0.9307 - val_accuracy: 0.7159
Epoch 6/15
209/209 [==============================] - 3s 13ms/step - loss: 0.9087 - accuracy: 0.7091 - val_loss: 0.9127 - val_accuracy: 0.7198
Epoch 7/15
209/209 [==============================] - 3s 13ms/step - loss: 0.9050 - accuracy: 0.7127 - val_loss: 0.9252 - val_accuracy: 0.7116